TO DO:
1.  Access to the number of transposons (Insertions) in the upstream and downstream 3KB regions of every gene.
    - Count how many insertions are 300bp after the End location and before it.
2.  Compute the average transposon density per library.
    - Compute the total number of transposons in the genome over the length of the genome.
3.  Compare if the number of transposons in the upstream and downstream 3KB regions of every gene is less than the average transposon density of the library * 3KB , then discard the gene for analysis of essentiality.
    - if the number of insertions of step1 < average transposon density * 3KB , discard the gene for analysis of essentiality.

- import dataset : postproccessed_data_all_backgrounds.xlsx
- compute transposon density : total bp/total insertions
- compute number of insertions in the upstream and downstream 3KB regions of every gene , Feature_type=Gene; Dubious

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict


In [53]:
data_all=pd.read_excel('../postprocessed-data/postprocessed_data_all_backgrounds.xlsx',
engine='openpyxl')

In [54]:
## Refine data 
data_all.index=np.arange(0,len(data_all))
data_all.drop(columns=['Unnamed: 1'],inplace=True)
data_all.fillna(0,inplace=True)
data_all.rename(columns={'Unnamed: 0':'background'},inplace=True)

keys=['dbem3_b',
 'dnrp1_b',
 'bem1-aid_a',
 'dnrp1_a',
 'dbem1dbem3_b',
 'wt_merged',
 'dbem1dbem3_a',
 'bem1-aid-dbem3_a',
 'bem1-aid-dbem3_b',
 'wt_b',
 'wt_a',
 'dnrp1_merged',
 'bem1-aid_b',
 'dbem3_merged',
 'dbem3_a']

In [55]:
indexes_back=[] # indexes for the start of each background

for i in keys:
    indexes_back.append(np.where(data_all.loc[:,"background"]==i)[0])


In [56]:
# Filling the name of the background according to the index
for k in np.arange(0,len(indexes_back)-1):
    
    data_all.loc[np.arange(indexes_back[k][0],indexes_back[k+1][0]),"background"]=keys[k]

data_all.loc[np.arange(indexes_back[-1][0],len(data_all)),"background"]=keys[-1] # for the last key

In [57]:
# Check
data_all.groupby(['background']).size()

background
bem1-aid-dbem3_a    12760
bem1-aid-dbem3_b    12760
bem1-aid_a          12745
bem1-aid_b          12760
dbem1dbem3_a        12760
dbem1dbem3_b        12760
dbem3_a             12760
dbem3_b             12760
dbem3_merged        12760
dnrp1_a             12745
dnrp1_b             12760
dnrp1_merged        12760
wt_a                12760
wt_b                12760
wt_merged           12745
dtype: int64

In [58]:
# Compute average transposon density per background 
data_all_processed=data_all.copy()
for key in keys:
    sum_tr=data_all_processed[data_all_processed.loc[:,"background"]==key]["Ninsertions"].sum()
    genome=data_all_processed[data_all_processed.loc[:,"background"]==key]["Nbasepairs"].sum()
    data_all_processed.loc[data_all_processed.loc[:,"background"]==key,"transposon_density"]=sum_tr/genome

In [89]:
# Threshold for the number of transposons in 3kB = 3000bp according the transposon density per background

threshold_tr=defaultdict(dict)
windows_size=3000
for key in keys:
    threshold_tr[key]["threshold"]=np.round(data_all_processed[data_all_processed.loc[:,"background"]==key]["transposon_density"].unique()[0]*windows_size,decimals=0)

In [90]:
threshold_tr_pd=pd.DataFrame.from_dict(threshold_tr,orient='index')

In [91]:
threshold_tr_pd

,threshold
bem1-aid-dbem3_a,82.0
bem1-aid-dbem3_b,122.0
bem1-aid_a,30.0
bem1-aid_b,61.0
dbem1dbem3_a,123.0
dbem1dbem3_b,116.0
dbem3_a,148.0
dbem3_b,159.0
dbem3_merged,248.0
dnrp1_a,115.0
